In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [6]:
#Import distance csv - to keep track of expected crack data
distance_df = pd.read_csv("../data/finalized_data/combined_distance_table.csv") #distance data should have all cracks
print(len(distance_df))
bridge_participant_cracks = {bridge: {} for bridge in ["Bridge 1", "Bridge 2", "Bridge 3", "Bridge 4"]}

#Extract a number from crack name
def extract_number(aoi_label):
    match = re.search(r'\d+', aoi_label)  # Find the first sequence of digits
    return int(match.group()) if match else None  # Convert to int if found

for index, row in distance_df.iterrows():
    bridge = row["Study Name"]
    participant = row["Respondent Name"]
    label = row["Label"]
    if "crack" in label.lower():
        if participant not in bridge_participant_cracks[bridge]:
            bridge_participant_cracks[bridge][participant]=set()
        
        cracknumber = extract_number(label)
        bridge_participant_cracks[bridge][participant].add(cracknumber)
    
print(bridge_participant_cracks)
    

2061
{'Bridge 1': {20002: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, 20004: {1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17, 18, 19, 20}, 20005: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, 20008: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, 20009: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20}, 20011: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, 20013: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20}, 20014: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, 20015: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}, 20016: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20}, 20017: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18, 19, 20}, 20021: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20}, 20022: {1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}, 20024: {1, 2, 4, 5, 6, 7

It's commented below bc i changed the implementation and they are no longer needed

In [4]:
#Read your aggrevated data file here
# data_df = pd.read_csv("data/finalized_data/saccade_final_3seconds.csv")
# # data_df = pd.read_csv("data/saccade_processing/combined_no_outlier_saccade_table.csv")
# data_df = pd.read_csv("../data/saccade_processing/combined_filtered_SaccadeTable_3seconds.csv")
data_df = pd.read_csv('../data/finalized_data/sensor/result_sensor_data_3seconds_2.csv')

existing_bridge_participant_cracks = {bridge: {} for bridge in ["Bridge 1", "Bridge 2", "Bridge 3", "Bridge 4"]}

#Extract a number from crack name
def extract_number(aoi_label):
    match = re.search(r'\d+', aoi_label)  # Find the first sequence of digits
    return int(match.group()) if match else None  # Convert to int if found

for index, row in data_df.iterrows():
    participant = int(row["Respondent"]) #Feel free to edit
    aoi_label = row["AOI Label"] #Feel free to edit
    bridge = row["Bridge"] #Feel free to edit

    if "base" in aoi_label.lower():
        continue

    if "crack" in aoi_label.lower():
        crack_number = extract_number(aoi_label)
        if participant in existing_bridge_participant_cracks[bridge]:
            existing_bridge_participant_cracks[bridge][participant].add(crack_number)
        else:
             existing_bridge_participant_cracks[bridge][participant] = set()
             existing_bridge_participant_cracks[bridge][participant].add(crack_number)
print(data_df)
#This is what we have in your current data (existing cracks)
# print(existing_bridge_participant_cracks)

        Bridge  Respondent     AOI Label  Mean Pupil Diameter Change  \
0     Bridge 1       20002   Crack 1 Hit                    1.055991   
1     Bridge 1       20002   Crack 2 Hit                   -0.025277   
2     Bridge 1       20002   Crack 4 Hit                    0.266140   
3     Bridge 1       20002   Crack 5 Hit                    0.022548   
4     Bridge 1       20002  Crack 12 Hit                    0.836557   
...        ...         ...           ...                         ...   
1987  Bridge 4       20017   Crack 9 Hit                   -0.044980   
1988  Bridge 4       20017  Crack 12 Hit                    0.157313   
1989  Bridge 4       20017  Crack 10 Hit                    0.061257   
1990  Bridge 4       20017  Crack 14 Hit                    0.405586   
1991  Bridge 4       20017  Crack 13 Hit                    0.737726   

      Median Pupil Diameter Change  STD Pupil Diameter Change  \
0                         1.071121                   0.372531   
1    

In [7]:
#Compare expected data and existing data and see what's missing
missing_participants = {bridge: {} for bridge in bridge_participant_cracks}
for bridge in missing_participants:
    missing_participants[bridge] = set()

#Check if you are missing any participant data or if you have extra participant data
for bridge in bridge_participant_cracks:
    for participant in bridge_participant_cracks[bridge]:
        if participant not in existing_bridge_participant_cracks[bridge]:
            missing_participants[bridge].add(participant)
print("Missing participant: ", missing_participants) #should be all empty set

extra_participants = {"Bridge 1": [], "Bridge 2": [], "Bridge 3": [], "Bridge 4": []}
for bridge in existing_bridge_participant_cracks:
    for participant in existing_bridge_participant_cracks[bridge]:
        if participant not in bridge_participant_cracks[bridge]:
            extra_participants[bridge].append(participant)
print("Extra participant you have: ", extra_participants)
      
#Check if you are missing any crack per participant
expected_count = 0
existing_count = 0
missing_cracks_per_participant = {bridge: {} for bridge in bridge_participant_cracks}
extra_cracks_per_participant = {bridge: {} for bridge in bridge_participant_cracks}
for bridge in bridge_participant_cracks:
    for participant in bridge_participant_cracks[bridge]:
        if participant in existing_bridge_participant_cracks[bridge]:
            expected_entries = bridge_participant_cracks[bridge][participant]
            existing_entries = existing_bridge_participant_cracks[bridge][participant]
            expected_count +=len(expected_entries)
            existing_count += len(existing_entries)
            missing_cracks = expected_entries-existing_entries
            extra_cracks = existing_entries-expected_entries
            missing_cracks_per_participant[bridge][participant] = missing_cracks
            extra_cracks_per_participant[bridge][participant] = extra_cracks

    
print("Missing cracks per participant: ")
for bridge in missing_cracks_per_participant:
    print(bridge)
    for participant in missing_cracks_per_participant[bridge]:
        print(participant, " : ", missing_cracks_per_participant[bridge][participant])
    
print("Extra cracks you have: ") 
for bridge in extra_cracks_per_participant:
    print(bridge)
    for participant in extra_cracks_per_participant[bridge]:
         print(participant, " : ", extra_cracks_per_participant[bridge][participant])
print(existing_count)
print(expected_count)
print("Final ratio: ", existing_count/expected_count)
print(len(data_df))

Missing participant:  {'Bridge 1': {20028}, 'Bridge 2': set(), 'Bridge 3': set(), 'Bridge 4': set()}
Extra participant you have:  {'Bridge 1': [], 'Bridge 2': [], 'Bridge 3': [], 'Bridge 4': []}
Missing cracks per participant: 
Bridge 1
20002  :  set()
20004  :  set()
20005  :  set()
20008  :  set()
20009  :  {6, 7, 8, 9, 10, 11, 12, 17, 18, 19, 20}
20011  :  set()
20013  :  set()
20014  :  set()
20015  :  set()
20016  :  set()
20017  :  set()
20021  :  set()
20022  :  set()
20024  :  set()
20025  :  set()
20026  :  set()
20027  :  set()
20029  :  set()
20030  :  set()
20031  :  set()
20032  :  set()
20034  :  {1, 2, 4, 5, 7, 8, 12, 17}
20035  :  set()
20036  :  set()
20037  :  set()
20038  :  set()
20039  :  set()
20041  :  set()
20042  :  set()
20043  :  set()
20044  :  set()
20045  :  set()
20046  :  set()
Bridge 2
20002  :  {7}
20004  :  set()
20005  :  {8, 11, 6}
20006  :  set()
20008  :  set()
20009  :  set()
20010  :  set()
20012  :  set()
20013  :  set()
20015  :  set()
20016  